In [20]:
import os
from dolfinx import mesh, io
from mpi4py import MPI
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from dolfinx import geometry
from dolfinx.io import gmshio, XDMFFile, VTXWriter
from dolfinx.fem import functionspace, Function, Constant
from dolfinx.fem.petsc import LinearProblem
import ufl
from ufl import grad, inner, Measure
from dolfinx import log, default_scalar_type
import dolfinx.fem.petsc as dfx_petsc

os.chdir("/home/acoustics/")
from utils.acoustics_utils import MicrophonePressure
from utils.mesh_utils import read_xdmf_data
from utils.gmsh_step_mesher import load_pickle

mesh_folder = os.path.join(os.getcwd(), "meshes")
RESULTS = os.path.join(os.getcwd(), "results")


In [21]:
# Load mesh and data
PKL_PATH = "/home/acoustics/meshes/4_5_2p5_data.pkl"
DATA_DICT = load_pickle(PKL_PATH)
VOL_DICT = DATA_DICT["volume"]
ORIENTATION = DATA_DICT["orientation"]
XDMF_PATH = DATA_DICT["xdmf_path"]

In [22]:
gdim = 3
domain, ct, ft = read_xdmf_data(xdmfPath=XDMF_PATH, comm=MPI.COMM_WORLD, gdim=gdim)
domain.topology.create_connectivity(0, domain.topology.dim) 
deg = 2
V = functionspace(domain, ("CG", deg))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Discrete frequency range
freq = np.arange(10, 1000, 5)  # Hz

# Air parameters
rho0 = 1.225  # kg/m^3
c = 340  # m/s


In [23]:
# Impedance calculation
def delany_bazley_layer(f, rho0, c, sigma):
    X = rho0 * f / sigma
    Zc = rho0 * c * (1 + 0.0571 * X**-0.754 - 1j * 0.087 * X**-0.732)
    kc = 2 * np.pi * f / c * (1 + 0.0978 * (X**-0.700) - 1j * 0.189 * (X**-0.595))
    Z_s = -1j * Zc * (1 / np.tan(kc * d))
    return Z_s


sigma = 1.5e4
d = 0.01
Z_s = delany_bazley_layer(freq, rho0, c, sigma)

In [24]:
omega = Constant(domain, default_scalar_type(0))
k = Constant(domain, default_scalar_type(0))
Z = Constant(domain, default_scalar_type(0))
v_n = 1e-5

In [25]:
ds = ufl.Measure("ds", domain=domain, subdomain_data=ft)

In [26]:
p = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = (
    ufl.inner(ufl.grad(p), ufl.grad(v)) * ufl.dx
    + 1j * rho0 * omega / Z * ufl.inner(p, v) * ds(VOL_DICT[1][0])
    - k**2 * ufl.inner(p, v) * ufl.dx
)
L = -1j * omega * rho0 * ufl.inner(v_n, v) * ds(VOL_DICT[1][1])

In [27]:
p_a = Function(V)
p_a.name = "pressure"

problem = LinearProblem(
    a,
    L,
    u=p_a,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
)

p_mic = np.zeros((len(freq), 1), dtype=complex)

mic = np.array([0.5, 0.05, 0.05])
microphone = MicrophonePressure(domain, mic)

In [ ]:
from dolfinx.io import VTXWriter
import os

t0 = 0.0
t1 = 1
dt = 0.01
num_time_steps = int((t1 - t0) / dt)


results = "/home/acoustics/results"
for nf in range(0, len(freq)):
    k.value = 2 * np.pi * freq[nf] / c
    omega.value = 2 * np.pi * freq[nf]
    Z.value = Z_s[nf]

    time_function = Function(V)
    time_function.name = "pressure"

    pressure_real = p_a.x.array.real
    pressure_imag = p_a.x.array.imag
    
    problem.solve()
    p_a.x.scatter_forward()
    # with VTXWriter(MPI.COMM_WORLD, os.path.join(results, f"pressure_output_{freq[nf]}.bp"), [p_a]) as writer:
    #     writer.write(0.0)

    with VTXWriter(MPI.COMM_WORLD, os.path.join(results, f"pressure_field_{freq[nf]}.bp"), [time_function]) as writer:
        for step in range(num_time_steps):
            t = t0 + step * dt
            time_factor_value = np.exp(1j * omega.value * t)
            timestepped = pressure_real * np.cos(omega.value * t) - pressure_imag * np.sin(omega.value * t)
            time_function.x.array[:] = timestepped
            time_function.x.array[:] = time_function.x.array.real
            time_function.x.scatter_forward()
            writer.write(t)

    p_f = microphone.listen(p_a)
    p_f = domain.comm.gather(p_f, root=0)

    if domain.comm.rank == 0:
        assert p_f is not None
        p_mic[nf] = np.hstack(p_f)